# Workbook 6: Supervised Machine Learning

## Description and aims

This tutorial is designed to give you your first experience of machine learning in practice by implementing a simple nearest-neighbour classifier.

The learning outcomes are:
- experience of implementing the K Nearest Neighbours classification algorithm
- experience of using the sklearn DecisionTree classification algorithm
-  experience of working through different preprocessing steps to try and improve the performance of your classifier

<div class="alert alert-warning">
    <h1>Activity 1: Loading and Visualising Data</h1>
   We will start by importing and visualising the two datasets used as examples in the lecture: students marks,  and Iris
<ul>
    <li>You should already have uploaded the data and figures from the lecture materials folder - if not, do so now.</li>
    <li>Then run the 5 code cells below to load and display the two datasets</li>
            </ul></div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def show_scatterplot_matrix(X,y,featureNames,title=None):
    f = X.shape[1]
    if(len(y) != X.shape[0]):
        print("Error,   the y array  must have the same length as there are rows in X")
        return
    fig, ax = plt.subplots(f,f,figsize=(12,12))
    plt.set_cmap('jet')
    for feature1 in range(f):
        ax[feature1,0].set_ylabel( featureNames[feature1])
        ax[0,feature1].set_xlabel( featureNames[feature1])
        ax[0,feature1].xaxis.set_label_position('top') 
        for feature2 in range(f):
            xdata = X[:,feature1]
            ydata = X[:,feature2]
            ax[feature1, feature2].scatter(xdata,ydata,c=y)
    if title != None:
        fig.suptitle(title,fontsize=16,y=0.925)
        
        
# simple function - currently only works for 2D data - but could easily be extended
def PlotDecisionSurface(trainX,trainy,theClassifier,theTitle,featureNames,xvar=0,yvar=1,stepSize=2.0,minZero=False):
    #create and prettify the plot
    cmap="Set3"
    fig,ax= plt.subplots(figsize=(8, 8))
    ax.set_title(theTitle)
    ax.set_xlabel(featureNames[xvar])
    ax.set_ylabel(featureNames[yvar])

    #define a grid we use to plot the decision boundaries
      #get max/min values for gri edges
    columnMax,columnMin = np.max(trainX,axis=0), np.min(trainX,axis=0)
    if(minZero==True):
        x_min , y_min= 0,0
    else:
        x_min, y_min = columnMin[ xvar]*0.95, columnMin[yvar]*0.95
    x_max, y_max = columnMax[xvar]*1.05, columnMax[yvar]*1.05 
    #make the grid
    xx, yy = np.meshgrid(np.arange(x_min, x_max, stepSize),np.arange(y_min, y_max, stepSize))

    #predict and plotfor evey point on the grid
    Z = theClassifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z,cmap=cmap)

    # Plot also the training points
    ax.scatter(x=trainX[:,xvar ],y= trainX[:, yvar], c=trainy.astype(float), alpha=1.0, cmap=cmap, edgecolor="black")
            

### The Student marks dataset

In [ ]:

grades= np.genfromtxt("../lectures/data/assessment-grades-2features.csv", delimiter= ',',skip_header=1)

featureNames=("exam", "CW_mean")
nStudents = grades.shape[0]

outcomes= ("Pass","Resit Exam", "Resit Coursework","Resit Both")
simpleoutcomes= ("pass","resit")

# make target labels
result = np.empty(nStudents, dtype=np.int8)

for row in range (nStudents):
    exam = grades[row][0]
    cw   = grades[row][1]
    if (exam>=35 and cw>=35 and (exam +cw >=80) ):
        result[row] = 0 # PASS 

    elif ( cw>=40 and exam < 40):
        result[row] = 1 #resit just exam 
    elif ( cw<40 and exam>=40):
        result[row]= 2 # resit just coursework
    else:
        result[row] = 3  # resit both
        
simpleResult = np.where(result<1,0,1)

In [ ]:
# easiest to split the data into 4/2 subgroups ot plot the outomes /simplified outcomes

passStudents = np.empty((0,2))
resitCWStudents = np.empty((0,2))
resitExamStudents = np.empty((0,2))
resitBothStudents = np.empty((0,2))

for student in range (nStudents):
    if (result[student]==0):
        passStudents = np.vstack( (passStudents,grades[student]) )
    elif (result[student]==1):
        resitExamStudents = np.vstack( (resitExamStudents,grades[student]) )
    elif (result[student]==2):
        resitCWStudents = np.vstack( (resitCWStudents,grades[student]) )
    else:
        resitBothStudents = np.vstack( (resitBothStudents,grades[student]) )
simpleResitStudents = np.vstack( (resitExamStudents,resitCWStudents,resitBothStudents))

print(passStudents.shape)
print(resitExamStudents.shape)
print(resitCWStudents.shape)
print(resitBothStudents.shape)
print(simpleResitStudents.shape)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(14,5))
plt.xlabel("Exam")
plt.ylabel("Coursework")
ax[0].set_title("Outcomes")
ax[1].set_title("Simplified Outcomes")

ax[0].scatter(passStudents[:,0],passStudents[:,1],label = "Pass" )
ax[0].scatter(resitExamStudents[:,0],resitExamStudents[:,1],label = "Resit Exam" )
ax[0].scatter(resitCWStudents[:,0],resitCWStudents[:,1],label = "Resit CW" )
ax[0].scatter(resitBothStudents[:,0],resitBothStudents[:,1],label = "Resit Both" )
ax[1].scatter(passStudents[:,0],passStudents[:,1],label = "Resit" )
ax[1].scatter(simpleResitStudents[:,0],simpleResitStudents[:,1],label = "Pass" )

ax[0].legend(loc='lower right')
ax[1].legend(loc='lower right') 

### Example 2:  Iris flowers <img src="../lectures/figures/ML/Iris-image.png" style="float:right">
- classic Machine Learning Data set
- 4 measurements: sepal and petal width and length
- 50 examples  from each 3 sub-species for iris flowers
- three class problem:
 - so for some types of algorithm have to decide whether to make  
   a 3-way classifier or nested 1-vs-rest classifers
- most ML classifiers can get over 90%



In [ ]:
import sklearn.datasets
irisX,irisy = sklearn.datasets.load_iris(return_X_y=True)
columnLabels= ("sepal_length", "sepal_width", "petal_length", "petal_width")
title="Scatterplots of 2D slices through the 4D Iris data"
show_scatterplot_matrix(irisX,irisy,columnLabels,title)

<div class="alert alert-warning">
    <h1>Activity 2: Implementing K-Nearest Neighbours</h1>
</div>
            
Basic process for predicting the label of a new point from the trainig set
1. Measure distance to new poitn from every member of the trainig set
2. Find the K Nearest Neighbours  
   in other words, the K members of the trainig set with the smallest distances  (*calculated in step 1*)
3. Count the labels that were provided for those K trainig items,  
   and return themost common one as the predicted label.

Below is a figure illustrating the start and first two steps of process.  
It is followed by a code cell with a simple implementation of a class for 1-Nearest neighbours. 

Read through the code  to get a sense for how it implements the algorithm.  
Your tutor will discuss it with you in the lab sessions.
<img src="../lectures/figures/ML/kNN-steps.png">



In [6]:
# Example for K = 1 

class simple_1NN:

    def __init__(self,verbose = True):
        # this version only looks at the single nearest neighbour
        self.K=1
        self.verbose= verbose
        
    def fit(self,X,y):
        # ask the data how big it is and store that info
        self.numExemplars = X.shape[0]
        self.numFeatures = X.shape[1]
        # store a copy of the data (X) and the labels (y)
        self.modelX = X
        self.modelY = y
        self.labelsPresent = np.unique(self.modelY) # list the unique values found in the labels provided
        if (verbose):
            print("There are {} training examples, each described by values for {} features".format(self.numExemplars,self.numFeatures))
            print("So self.modelX is a 2D array of shape {}".format(self.modelX.shape))
            print("self.modelY is a list with {} entries, each being one of these labels {}".format(len(self.modelY), self.labelsPresent))
        
    def predict(self,newItems):
        # read how many  newitems there are
        numToPredict = newItems.shape[0]
        # make an empty list to hold their predicted labels
        predictions = []
        
        #loop through each new item each one
        for item in range(numToPredict):
            # predicting its label
            thisOPrediction = self.PredictNewItem ( newItems[item])
            # adding that predictin to our list
            predictions = predictions.append(thePrediction)
        return predictions
    
    def PredictNewItem(self,newItem):
        
        # Step 1: measure and store distance to each training item
        distFromNewItem = np.zeros((self.numExemplars)) # array with one entry for each trainig set item, intialised to zero
        for exemplar in range (self.numExemplars):
            distFromNewItem[exemplar] = EuclideanDistance(newItem,  self.modelX[exemplar])
  
        # Step 2: find the one closest training example: This is K=1, 
        closest = 0
        for trainingExample in range (0, self.numExemplars):
            if  ( distFromNewItem[trainingExample] < distFromNewItem[closest] ):
                closest=trainingExample
 
        # step 3: count the votes - because this is for K=1 so we don't need to take a vote
        labelOfClosest = self.modelY[closest]
        return labelOfClosest
    
    def EuclideanDistance(a,b):
        ## this numpy function calculates the euclidean distance
        return np.linalg.norm(a-b)
  


<div class="alert alert-warning" >
<h2> Activity 2.1</h2>
    Run the code provided for K=1 with the two datasets and make sure you understand the outputs and how they are produced
<ul>
    <li>For the marks dataset this creates a plot to show a decision surface</li>
    <li>For the  iris data set this uses a confusion matrix <br> (google what a confusion matrix is if you're not sure)</li>
    </ul>
    </div>

**The Marks dataset - illustrating a 2D Decision surface**

In [ ]:



# create and train the classifier
myKNNmodel = simple_1NN()
myKNNmodel.fit(grades,simpleResult) 

#visualise the decision surface
PlotDecisionSurface(grades, simpleResult, myKNNmodel,"1-NN simplified outcomes", ("exam","cw"),minZero=True)

**The Iris dataset - illustrating a confusion matrix**

In [ ]:
# make train/test split 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

irisX,irisy = load_iris(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(irisX, irisy, test_size=0.33,stratify=irisy)


model = simple_1NN()
model.fit(X_train,y_train)
ypred = model.predict(X_test)
confusionMatrix = np.zeros((3,3),int)
for i in range(50):
    actual = int(y_test[i])
    predicted = int(ypred[i])
    confusionMatrix[actual][predicted] += 1
print(confusionMatrix)



<div class="alert alert-warning" >
<h2> Activity 2.3: Create your own implementation of K-Nearest Neighbours</h2>
    Using the code above,  extend the predict method for the class simple_1NN  to use the votes from K>1 neighbours.


<ul>
    <li>Start by creating an empty class called Simple_KNN and copying in the pseudo-code as comments</li>
    <li>Then copy the code from the simple_1NN class into the relevant places</li>
    <li> You should only need to make minor changes to the __init__ method to set the value of K </li>
    <li> in the predictNewItem() method you will need to change step 2  and step 3 </li>
    <li> In the pseudocode you were suggested two possible ways of doing step 2. </li>
    <li> If you choose to do step 2 the <emph>pythonic</emph> way, the next markdown cell discusses how to do this and the following code cell gives an example.  </li>
    </ul>


### Pseudocode for KNearest Neighbours
**init()**  :  
SPECIFY function to calculate distance metric d(i,j) for any two items *i* and *j*     
  e.g. Euclidean (continuous variables) or Hamming (categorical)  
SET value of K

**fit(trainingData)** :  

SET numExemplars = READ(number of rows in training data)  
SET numFeatures = READ(number of columns in training data) 

*#Just store a local copy of the training data as two arrays:*   
CREATE_AND_FILL(X_train of shape (numExemplars , numFeatures)).     
CREATE_AND_FILL(y_train of shape( numExemplars))
  
**predict(newItems)** :  
SET numToPredict = READ(number of rows in newItems)  
SET predictions = CREATE_EMPTYARRAY( numToPredict)
 
FOREACH item in (0...,numToPredict-1)    
...SET predictions[item] = predictNewItem ( newItems[item]) 
 
RETURN predictions  


**predictNewItem(newItem)**:

*Step 1:   Make 1D array distances from newItem to each trainig set item*   
FOREACH exemplar in (0,...,numExemplars -1  
...SET distFromNewItem [exemplar] = d (newItem , X_train[exemplar] )   

*Step 2: Get indexes of the k nearest neighbours for our new item*        
SET closestK = GET_IDS_OF_K_SMALLEST(distFromNewItem)
 
  
*Step 3: Store majority vote in a  1D array y_pred with numToPredict entries*     
SET labelcounts = CREATE(1D array with m zero values)  

FOREACH  k in (0,...K-1)   
... SET thisindex = closestK[newItem][k]  
... SET thislabel = y_train[thisindex]  
... INCREMENT labelcounts[thislabel]  

SET thisPrediction = READ(index of labelcounts with highest value)    

RETURN thisPrediction

## Two possible Versions of GET_IDS_OF_K_SMALLEST for  Step 2 

**V1: The "pythonic" way using sorting**  
SET distFromNewItemWithIndex = CREATE(2D array with  2 columns, and numExemplars rows)  
FOREACH exemplar in (0,...,numExemplars -1)    
...SET distFromNewItemWithIndex[exemplar][0] = distFromNewItem[exemplar] *#column 0 holds distance*   
...SET distFromNewItemWithIndex[exemplar][1] = exemplar          *#column 1 holds Id of the exemplar*  

SET  sortedByDist = SORT (rows of distFromNewItemWithIndex by increasing distances (values in column 0) )  

SET closestK = SELECT (first K elements from column 1 of  sortedByDist) # can do in one go using slicing
RETURN closestK 


**V2: More explicit version using loops**
SET closestK= EMPTYLIST

FOR k in (0,...,K-1)  
... SET thisClosest=0  
... FOR exemplar in (1,...,K-1)  
......IF ( distFromNewItem[exemplar] < distFromNewItem[thisClosest]  )  
......... SET thisClosest = exemplar  
... SET closestK[k] = thisClosest # store this id  
... SET distFromNewItem[thisClosest] = BigNumber # so we don't pick it again in next loop

RETURN closestK


In [ ]:
# your KNN class code here

class simple_KNN:
    

**Hints on how to do step 2 v1 - the 'pythonic' way**  


This may be slightly more efficient **but is more suited to people confident in coding**

If I have an array of myDist of (say, for simplicity) five values,  and I want to resort them by size, keeping track of the item ids  so I can find the K with the smallest values in the original array.

What I need to do is:
1. Create a new array with two columns - the distances in the first colum, and the Ids in ther second column
2. Resort the rows of this array in ascending order of the first column. 
   i.e. the row with the smallest ditacnes moves ot the top etc.
3. Then the second column now holds the index/ids sorted according to size of their contents

<img src="howto-get-K-closest.png" style="float:right" width = 50%>
The image shows the idea of how to do this.
- it starts by grabbing a row from a 2D distacnes array,    
  but the simple_1NN class shows how to calculate this row directly



   I can do it using the code below, which:
   - creates a 2d array **myUnsortedArray** holding each value and its index in the original array
   - then makes a new array **mySortedArray**  by sorting according to the values in the first column 
   - then looks in the second column (which holds the indexes) for the first K rows!

In [ ]:
# This cells just contains some hints about how to prodice a sorted array
myDist = np.array([4,7,1,3,9])

myUnsortedArray = np.empty((5,2))
print(myUnsortedArray.shape)
for row in range (5):
    myUnsortedArray [row][0] = myDist[row]
    myUnsortedArray [row][1] = row
print('myUnsortedArray contents before sorting')
print(myUnsortedArray)

mySortedArray=myUnsortedArray[np.argsort(myUnsortedArray[:, 0])]

print('mySortedArray contents after sorting')
print(mySortedArray)

print('Last week we learned about slicing- we will now use slicing to pull out all rows but just specific columns')
print(' the values in array myDist in ascending order are: {}'.format(mySortedArray[:,0]))
print(' the positions those values were in, again by ascending order of value are : {}'.format(mySortedArray[:,1]))
print('Now remember you do not have to choose every row ... ')

<div class="alert alert-warning" >
<h2> Activity 2.4: Test your implementation on the two example datasets</h2>
Use the toolbar to copy and paste the two cells from activity 2.1 below here. <br>
Then edit them so that they create and use objects of your new class, instead of the class simple_1NN

Start with K=1 - this should produce the same results as you got in activity 2.1, then try with K = {3,5,7}
<ul>
    <li>Use the student marks for <b>qualititative</b> judgements : how does the decision surface change?</li>
    <li>Use the Iris data set for <b>quantitative</b> judgements :  how does the confusion matrix change?</li>
    </ul>
    </div>

<div class="alert alert-warning" >
<h1> Activity 3: Decision Trees</h1></div>




<div class="alert alert-warning" >
<h2> Activity 3.1: Run the next cell to remind yourself how the decision trree model is grown</h2>

The next cell just illustrates how the tree induction process works for the student marks dataset.<br>
It just calls the decision tree repeatedly for increasing depths.<br>
For depth 0 I've just created a text box with the relevant stats in.</div>


In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

fig,ax = plt.subplots(1,3,figsize=(18,8))
fig.suptitle("Illustration of how Decision Trees select and insert nodes to increase data purity")
for depth in range (0,3):
    if(depth==0):
        ax[0].text(0.25, 0.6, " gini=0.147\n samples=150,\n value=[138,12],\n class=pass",fontsize=14, 
        bbox={'facecolor': 'darkOrange', 'alpha': 0.5, 'pad': 10})
        ax[0].axes.get_yaxis().set_visible(False)
        ax[0].axes.get_xaxis().set_visible(False)
        ax[0].set_frame_on(False)
        ax[0].set_title("Depth 0")
    else:
        model = DecisionTreeClassifier(random_state=1234, max_depth=depth,min_samples_split=2,min_samples_leaf=1)
        model.fit(grades,simpleResult)
        _ = tree.plot_tree(model, feature_names=("exam","coursework"), class_names= ("pass","resit"),filled=True,ax=ax[depth])
        ax[depth].set_title("Depth "+str(depth))
        
fig.savefig("DecisionTreeExample-studentMarks.png")

<div class="alert alert-warning" >
<h2> Activity 3.2: exploring how to control tree-growth to prevent over-fitting</h2>
The aim of this activity is for you to experiment with what happens when you change three parameters that affect how big and complex the tree is allowed to get.
<ul>
    <li> max_depth</li>
    <li>min_samples_split, (default value is 2)</li>
    <li>min_samples_leaf, (default value is 1)</li>
    </ul>


Experiment with the Iris data set below to see if you can work out what each of these parameters does, and how it affects the tree 
<ul>
<li> Each time you run the  cell below, it will give you a different train-test split of the Iris data.<br>
    Does this affect what tree you get? </li>
    <li> Is there a combination of values that means you consistently get similar trees?</li>
    <li>    What is a good way of judging 'similarity?</li>
    </ul>
    </div>

In [ ]:

# load iris dataset and split into train:test
iris = sklearn.datasets.load_iris()
irisX = iris.data
irisy = iris.target

X_train, X_test, y_train, y_test = train_test_split(irisX, irisy, test_size=0.33,stratify=irisy)

model = DecisionTreeClassifier(random_state=1234, max_depth=None,min_samples_split=2,min_samples_leaf=1)
model.fit(X_train,y_train)
ypred = model.predict(X_test)
confusionMatrix = np.zeros((3,3),int)
for i in range(50):
    actual = int(y_test[i])
    predicted = int(ypred[i])
    confusionMatrix[actual][predicted] += 1
print(confusionMatrix)



fig = plt.figure(figsize=(12,12))
_ = tree.plot_tree(model, 
                   feature_names=iris.feature_names,  
                   class_names=iris.target_names,
                   filled=True)


<div class="alert alert-warning" > <h1> Activity 4: (stretch)</h1></div>
Using the code from last week,  apply a StandardScaler to the Iris data set and evaluate the effect this has on the accuracy.

Because there is a random element in how  the data set is split into training / test split,  it is not valid just to split the data once then compare the results with / without scaling.

Instead  you will need to do ten repeats  of:
- Use the sklearn method to split the data into 66:34 train/test sets
- Construct,  train, and test,  an instance of your kNN model on the unscaled data and store its accuracy 
- Create an instance of the standard scaler and then:
  - call its fit() method to set its parameters from the training set.
  - call its transform() method for both the traing and test sets
  - Construct,  train, and test,  an instance of your kNN model on this scaled data and store its accuracy 

That should gives you ten pairs of values (one per repeat) for the scaled and raw data accuracy.  
Use an online statistical tool (e.g. https://www.graphpad.com/quickcalcs/ttest1.cfm) that lets you copy your data in the perform a 'paired t-test" to find out the probability that normalising the data improves prediction accuracy

<div class="alert alert-block alert-danger"> Please save your work (click the save icon) then shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook</div>

<div class="alert alert-block alert-danger"> Remember to download and save your work if you are not running this notebook locally.</div>